<a href="https://colab.research.google.com/github/82KJ/ARS-system-for-protecting-telephone-counselor/blob/main/SH/HomeWork/Make_Final_Dict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 준비
- cpu 환경은 너무 느려서 코랩에서 부탁드립니다 ㅎ
- 꼭 GPU 가속이 켜져있는지 확인해주세요!
- 추가로, 구글 드라이브에 학습 데이터, 사전 데이터, 모델을 모두 로드해 주세요!

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-hsmjt4ce
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-hsmjt4ce
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 426.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.9 MB/s eta 0:00

In [3]:
! pip install kiwipiepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 54.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.15.0-py3-none-any.whl size=30602642 sha256=c1a552855bf24e0e23f93a3d607f426876af06842feab69d06ca174e094a4b71
  Stored in directory: /root/.cache/pip/wheels/9e/34/31/a1682c8249bf2ba89a29498d62c35c08ceb116537b4530d93e
Successfully built kiwipiepy-model


In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# 2. 데이터 준비
- 여기서는 training_dataset.csv를 활용해서 train, test set을 준비하게 됩니다!
- 제가 디렉토리에 같이 넣어놀테니까 이거 활용하세요! --> 구글 드라이브로 옮기기
- 추가로, 사전도 미리 준비해 놓겠습니다!

In [6]:
cd /content/drive/MyDrive/ARS-system-for-protecting-telephone-counselor/SH/HomeWork

/content/drive/MyDrive/ARS-system-for-protecting-telephone-counselor/SH/HomeWork


In [7]:
import pandas as pd
data = pd.read_csv("training_dataset.csv")

data_list = list()
for sen, lab in zip(data["0"], data["1"]):
  data_list.append([sen,lab])

In [8]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(data_list, test_size=0.1, random_state=0) # train : test = 9:1
train_set[0]

# 여기서 만들어진 test_set을 중점으로 활용하시면 됩니다!

['그 동안 보빨러들이 얼마나 잘 해줬겠어ㅋㅋㅋ', 2]

In [9]:
test_set[:3]

# 아래를 보면 알겠지만, 각 row마다 0번이 문장 데이터, 1번이 라벨링 데이터입니다!

[['인정 쟤네 목소리 쫌 심각하게 족같아', 1],
 ['나 알바하는 곳 너무 장사 잘돼서 힘들어.', 0],
 ['아슬아슬~ 정말 프로답게 벗어 주네.', 2]]

In [10]:
abuse = pd.read_csv("폭언사전.csv")
sexual = pd.read_csv("성희롱사전.csv")

# 사전은 아래에서 쓰일 거라서 미리 로드합니다!

In [11]:
abuse.head(3)

,Unnamed: 0,0
0,0,가난
1,1,가난뱅이
2,2,가두


In [12]:
sexual.head(3)

# 사전이 dataframe으로 로드되면서, 컬럼명[Unnamed:0, 0]으로 자동 지정되었어요!
# 여기서 '0' col을 사용하면 되겠죠?

,Unnamed: 0,0
0,0,19금
1,1,69
2,2,69자세


In [13]:
abuse_list = list()
sexual_list = list()

for rows in abuse["0"]:
  abuse_list.append(rows)

for rows in sexual["0"]:
  sexual_list.append(rows)

# dataframe로 load한 사전이 사용하기 불편할 수도 있으니, 리스트 형식으로도 저장합니다

In [14]:
abuse_list[:3], sexual_list[:3]

(['가난', '가난뱅이', '가두'], ['19금', '69', '69자세'])

# 3. 코버트 모델 로드
- 여기서는 저희가 만든 "kobert_classifier.pth"을 로드해서 모델을 준비합니다
- 모델은 가지고 계시죠? 그거 구글 드라이브에 로드해서 사용하시면 됩니다!

In [15]:
device = torch.device("cuda:0")

class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):

    # sentence , label data를 BERT의 입력값에 맞게 변환하는 transformer를 생성
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_len, pad=pad, pair=pair)

    ## 생성한 transformer로 sentence를 변환하여 저장
    self.sentences = [transform([data[sent_idx]]) for data in dataset]
    self.labels = [np.int32(data[label_idx]) for data in dataset]
  
  def __getitem__ (self, i):
    return (self.sentences[i] + (self.labels[i], )) # 각 index에 맞는 item 반환 진행 --> 왜 이런 형태인지는 잘 모르겠음
  
  def __len__(self):
    return (len(self.labels))

# Parameter setting 진행
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

# Kobert 모듈에서 제공하는 get_tokenizer와 vocab를 활용해 tokneizer를 구성한다
tokenizer = get_tokenizer() 
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_set, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_set, 0, 1, tok, max_len, True, False)

class BERTClassifier(nn.Module):
  def __init__(self, bert, hidden_size = 768, num_classes=3, dr_rate=None, params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    ## classifier는 선형 회귀 모델로 구성 (input size = 768, output size = 3 (label이 3개로 구성))
    self.classifier = nn.Linear(hidden_size, num_classes)

    ## overfitting 방지를 위한 dropout 비율 설정
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  # attention mask sequence를 구성해주는 함수 --> padding이 아닌 영역을 0에서 1로 변경
  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i,v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    
    return attention_mask.float()
  
  # bert + classifier를 관통하는 forward 연산 진행
  def forward(self, token_ids, valid_length, segment_ids):

    # attention_mask 계산
    attention_mask = self.gen_attention_mask(token_ids, valid_length)

    # bert에 input 투입, 변수명이 pooler인거 보니 출력 embedding에 mean pooling 적용한 값이지 않을까 추측
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))

    # dropout 비율이 존재한다면, dropout 적용
    if self.dr_rate:
        out = self.dropout(pooler)

    # classifier 진행
    return self.classifier(out) 

model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model_state_dict = torch.load("kobert_classifier.pth", map_location=device)
model.load_state_dict(model_state_dict)

using cached model. /content/drive/MyDrive/ARS-system-for-protecting-telephone-counselor/SH/HomeWork/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


<All keys matched successfully>

# 4. test_set 예측 라벨 산출
- 이제 3870개의 test 문장을 모델에 투입해서, 예측된 라벨링을 얻을 거예요!
- 결과 라벨은 y_hat에 저장할게요~

In [16]:
def predict(predict_sentence):
  # 1. data set 구성 (문장, 라벨)
  data = [predict_sentence, '0']
  dataset_another = [data]

  # 2. data를 bert의 입력에 맞게 변환하기
  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=0)
  
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)

      # 모델 forward 연산 진행
      out = model(token_ids, valid_length, segment_ids)
      
      # torch out -> numpy 형식으로 변환
      logits = out[0].detach().cpu().numpy()

      return np.argmax(logits)

In [17]:
y_hat = list()

for rows in test_set:
  labels = predict(rows[0])
  y_hat.append(labels)

In [18]:
y_hat[:3]

# 이게 저희 모델로 예측한 라벨 값들 입니다!

[1, 0, 2]

In [19]:
y_hat

[1,
 0,
 2,
 2,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 2,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 2,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 1,
 2,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 1,
 2,
 0,
 0,
 2,
 2,
 2,
 1,
 0,
 1,
 1,
 1,
 2,
 0,
 1,
 2,
 2,
 0,
 0,
 0,
 1,
 0,
 1,
 2,
 0,
 2,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 2,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 2,
 0,
 2,
 0,
 1,
 0,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 1,
 0,


In [20]:
y = list()

for rows in test_set:
  y.append(rows[1])

# test_set은 [문장, 실제 라벨값]의 구조라서 실제 라벨값만 따로 y에 저장했어요 

In [21]:
y[:3]

# 요건 실제 라벨값 입니다

[1, 0, 2]

In [ ]:
y

[1,
 0,
 2,
 2,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 2,
 0,
 1,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 1,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 2,
 0,
 1,
 2,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 2,
 1,
 0,
 1,
 2,
 0,
 2,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 2,
 1,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 2,
 0,
 2,
 0,
 1,
 0,
 0,
 2,
 1,
 0,
 2,
 0,
 0,
 1,
 0,


In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

cm = confusion_matrix(y, y_hat)
print("===Confusion Matrix===")
print(cm)

# y, y_hat으로 confusion matrix을 산출한 결과입니다
# row는 실제 라벨
# col은 예측 라벨
# 즉, 0행은 실제 일반 문장 1999개에서 (1912 + 54 + 33) 1912개는 올바르게 평가, 54개는 폭언으로 잘못 산출, 33개는 성희롱으로 잘못 산출 했다는 것을 의미합니다
# 우리의 목표는 54와 33으로 잘못 판별된 문장들을 모델 투입 전에 사전에서 미리 거를 수 있도록, 사전, 문장 각색을 진행하는 것입니다!

===Confusion Matrix===
[[1912   54   33]
 [  62  906   43]
 [  55   42  763]]


y_hat과 y 비교하여 다른 값 index_list

In [23]:
index_list = []

for i in range (len(y)):
  if y[i] != y_hat[i]:
    index_list.append(i)

print(index_list) #289
#print(len(index_list)) 
#근데 사실 이건 일반 아닌데 다르게 예측한 것도 포함되니깐 밑에 껄로 한다.

[18, 21, 24, 25, 33, 34, 93, 101, 104, 117, 132, 135, 139, 142, 150, 155, 159, 162, 171, 205, 213, 225, 246, 255, 270, 285, 300, 301, 309, 327, 336, 368, 387, 388, 400, 413, 414, 417, 423, 430, 466, 470, 492, 511, 518, 527, 569, 577, 591, 621, 622, 629, 651, 658, 672, 676, 677, 678, 681, 714, 715, 716, 747, 761, 774, 788, 798, 802, 807, 817, 824, 832, 835, 875, 882, 895, 907, 942, 945, 954, 961, 963, 964, 974, 990, 1002, 1010, 1047, 1058, 1067, 1071, 1079, 1091, 1099, 1102, 1105, 1108, 1149, 1168, 1204, 1219, 1233, 1238, 1267, 1310, 1315, 1332, 1371, 1372, 1389, 1390, 1391, 1396, 1436, 1441, 1456, 1457, 1472, 1501, 1510, 1521, 1528, 1531, 1537, 1563, 1593, 1632, 1638, 1642, 1674, 1696, 1697, 1704, 1736, 1745, 1750, 1761, 1773, 1777, 1786, 1810, 1815, 1819, 1832, 1840, 1872, 1884, 1909, 1935, 1940, 1974, 1996, 2034, 2039, 2040, 2059, 2078, 2079, 2092, 2097, 2105, 2114, 2115, 2143, 2197, 2203, 2208, 2217, 2225, 2245, 2249, 2267, 2297, 2310, 2312, 2353, 2368, 2371, 2378, 2392, 2394, 2397,

# 5. 사전 각색 or 문장 각색
- 이제 본격적인 알고리즘 작성이 필요합니다
- 대략적인 순서는 제가 여기에 적어둘게여
  - y 와 y_hat을 비교하면서, y==0 인데, y_hat != 0이 아닌 문장의 인덱스를 구합니다!
  - 이제, 구한 인덱스를 바탕으로 test_set에서 문장을 찾아서, 형태소 분해 후 저장을 합니다
  - 저장된 형태소 리스트를 바탕으로, 각각 사전의 어떤 키워드와 매칭되는지 확인합니다!
  - 마지막으로, 사전 각색 혹은 문장 각색 두 방법 중 하나를 선택해서 진행하시면 됩니다!


In [24]:
# 1. y 와 y_hat을 비교하면서, y==0 인데, y_hat != 0이 아닌 문장의 인덱스를 구합니다! (== 실제 일반 문장인데, 폭언, 성희롱으로 잘못 예측된 문장의 인덱스)

moral_abuse_idx = list() # 일반인데 폭언으로 잘못 판별된 문장의 인덱스 리스트
moral_sexual_idx = list() # 일반인데 성희롱으로 잘못 판별된 문장의 인덱스 리스트

for idx in range(len(y)):
  if y[idx] == 0 and y_hat[idx] == 1:
    moral_abuse_idx.append(idx)
  elif y[idx] == 0 and y_hat[idx] == 2:
    moral_sexual_idx.append(idx)


# 구한 인덱스의 길이가 confusion matrix의 54, 33으로 나오겠죠?
len(moral_abuse_idx), len(moral_sexual_idx)

(54, 33)

In [25]:
moral_abuse_idx

[33,
 104,
 135,
 171,
 246,
 368,
 413,
 417,
 577,
 672,
 677,
 716,
 798,
 961,
 974,
 1047,
 1204,
 1310,
 1332,
 1372,
 1396,
 1521,
 1528,
 1537,
 1697,
 1810,
 2040,
 2143,
 2203,
 2368,
 2451,
 2469,
 2471,
 2474,
 2564,
 2657,
 2742,
 2752,
 2909,
 2914,
 3027,
 3097,
 3167,
 3238,
 3343,
 3393,
 3477,
 3600,
 3626,
 3744,
 3746,
 3747,
 3789,
 3849]

In [26]:
moral_sexual_idx

[25,
 142,
 155,
 159,
 285,
 301,
 569,
 1091,
 1102,
 1168,
 1219,
 1389,
 1436,
 1642,
 1750,
 1761,
 2078,
 2097,
 2115,
 2267,
 2758,
 2765,
 2807,
 3074,
 3177,
 3341,
 3347,
 3375,
 3388,
 3532,
 3534,
 3749,
 3854]

In [27]:
# 2. 찾은 인덱스를 바탕으로 test_set에서 문장을 찾고, 형태소 분해 후 저장해요
from kiwipiepy import Kiwi
kiwi = Kiwi()

moral_abuse_morphs = list()    # 일반인데 폭언으로 잘못 판별된 문장의 형태소 리스트
moral_sexual_morphs = list()   # 일반인데 성희롱으로 잘못 판별된 문장의 형태소 리스트

#(1) 폭언
for idx in moral_abuse_idx: #폭언으로 잘못판별된 문장의 리스트 인덱스로
  morphs = kiwi.tokenize(test_set[idx][0]) #테스트 셋 문장을 찾고 형태소 분해
  temp = list()
  for morph in morphs:
    temp.append(morph.form)
  moral_abuse_morphs.append([idx,temp])

#(2) 성희롱
for idx in moral_sexual_idx:
  morphs = kiwi.tokenize(test_set[idx][0])
  temp = list()
  for morph in morphs:
    temp.append(morph.form)
  moral_sexual_morphs.append([idx,temp])

print(moral_abuse_morphs[0]) #이거는 잘못 판별된 애들
print(moral_sexual_morphs[0])

# 출력된 결과를 보면, 짱개, 성관계 등의 형태소가 포함되어 있어서 두 문장에 대한 표현은 사실 일반 문장이라고 보기 힘듭니다.. ㅜ
# 아래 문장들은 사실, 일반 문장 데이터 만들 때 제거되었어야 맞는데.. 흠.. 누군가 대충 넘어가서 이런 결과가.. ㅎ

[33, ['야', '짱개', '언어', '그냥', '모르', '어도', '되', '거든', '?']]
[25, ['성관계', '끝나', '었', '는데', '도', '계속', '이쁘', '다고', '하', '어', '주', '고', '이름', '이랑', '전화', '번호', '도', '알리', '어', '주', '네', '.']]


In [28]:
# 3. 우선, 폭언으로 잘못 판별된 문장들이 사전의 어느 부분이랑 매칭되는지 확인해볼까요?
# 이때, 사전은 폭언, 성희롱 사전 모두 사용하셔야 됩니다 --> 실제 시스템에서 두개다 매칭을 진행할거라서 ㅇㅇ


#잘못 판별된 애들이 사전에 어디랑 매칭되어 있는지 확인 
for rows in moral_abuse_morphs:
  abuse_matched = list()
  sexual_matched = list()

  for morph in rows[1]:
    if morph in abuse_list:
      abuse_matched.append(morph)
    if morph in sexual_list:
      sexual_matched.append(morph)

  rows.append(abuse_matched)
  rows.append(sexual_matched)

for rows in moral_abuse_morphs:
  print(rows)
  print()

# 폭언으로 잘못 분류된 문장 모두에 대해 (test_set 인덱스, 문장 형태소, 폭언 사전 매칭 형태소, 성희롱 사전 매칭 형태소)
# 보면 1396이랑,2752 인덱스는 어떤 사전이랑도 매칭되는 게 없잖아요. 바로 해당 문장들이 사전을 적용하게 되면, 매칭이 되는게 없어서 바로 일반 문장으로 판단되는 문장들입니다!
# 제가 말했었죠. 그 사전 적용하면 딱 2문장만 걸러진다고요. 그게 바로 이 두문장입니다!!
# 그러니까, 지금 저희가 하는 일이. 이 두문장 말고도 사전에 매칭되는 문장이 없게끔 문장, 사전을 각색하는 일입니다!

[33, ['야', '짱개', '언어', '그냥', '모르', '어도', '되', '거든', '?'], ['짱개', '어도'], ['어도']]

[104, ['걔', '네', '이름', '부터', '정', '떨어지', 'ᆷ', 'ㅋㅋ', '평생', '땅', '치', '고', '후회', '하', '었', '으면'], ['정', '떨어지', '땅', '치'], ['정', '으면']]

[135, ['안락사', '는', '죽음', '을', '존엄', '하', '게', '맞', '을', '수', '있', '게', '돕', '자는', '거', '이', 'ᆫ데', '.'], ['안락사', '죽음', '맞', '자는'], ['자는']]

[171, ['저번', '대선', '홍', '이', '문', '개털', '었', '는데', '?', '탄핵', '때문', '에', '지', 'ᆫ', '거', '이', '야'], ['홍'], ['는데']]

[246, ['너', '아이스크림', '먹', '고', '싶', '다며', '나', '가', '배', '민', '으로', '시키', '어', '주', 'ᆯ까', '?'], ['먹', '시키'], ['먹', '배', '시키', '주']]

[368, ['나', '그냥', '멍', '때리', '고', '있', '었', '어'], ['멍', '때리'], ['때리']]

[413, ['감방', '가', '어서', '도', '또', '사기치', '어', '먹', '을', '궁리', '하', '겠', '지', '점점', '더', '지능', '것', '으로', '크', 'ᆫ', '사기치력', '이', '고', '열심히', '연구', '하', '겠', '구만', '하', '는', '넘', '은', '또', '하', '지', '.'], ['감방', '사기치', '먹', '지능', '크', '넘'], ['어서', '먹', '크']]

[417, ['아니야', '장애', '있', '는', '사람', '들', '도', '결혼', '잘', '하', '어서', 

여기서부터 사전 각색

In [29]:
# 4. 사전 각색 방법

# 위에 문장 중에서, 171 인덱스 문장을 바탕으로 사전을 각색하는게 좋아보여요
# 폭언 사전에서는 '홍', 성희롱 사전에서는 '는데'만 매칭되었거든요
# 그럼 해당 키워드를 제거하기 전에, 기존에 '홍', '는데' 키워드만 가지고 있던 폭언, 성희롱 문장이 있을 수도 있어요
# 그런 문장이 있는지 확인해봐여

def isOnlyThatKeyword(keyword):
  res = list()

  for rows in test_set:
    if rows[1] == 1 or rows[1] == 2:  
      temp_all = list()
      temp_abuse = list()
      temp_sexual = list()
      morphs = kiwi.tokenize(rows[0])

      flag = False
      for morph in morphs:
        if morph.form == keyword:
          flag =True
      
      if flag == True:
        for morph in morphs:
          temp_all.append(morph.form)

          if morph.form in abuse_list:
            temp_abuse.append(morph.form)
          if morph.form in sexual_list:
            temp_sexual.append(morph.form)
        
      if len(temp_abuse) + len(temp_sexual) == 1:
        res.append([temp_all, temp_abuse, temp_sexual])
      

  return res


In [30]:
isOnlyThatKeyword('홍')

[]

In [31]:
isOnlyThatKeyword('는데') # 홍, 는데만으로 매칭되는 폭언, 성희롱 문장은 업네여!

[]

한번에 isOnlyThatKeyword하기

In [ ]:
temp_abuse_list = [] #매칭된 것 중에 폭언 매칭 한번에 담은 리스트
temp_sexual_list = [] #매칭된 것 중에 성희롱 매칭 한번에 담은 리스트
for rows in moral_abuse_morphs:
  for item in rows[2]:
    temp_abuse_list.append(item)
  
  for item in rows[3]:
    temp_sexual_list.append(item)

In [ ]:
idx_list = []
for rows in moral_abuse_morphs:
  for item in str(rows[0]):
    idx_list.append(item)

print(len(idx_list))


200


In [ ]:
temp_sexual_list

['어도',
 '정',
 '으면',
 '자는',
 '는데',
 '먹',
 '배',
 '시키',
 '주',
 '때리',
 '어서',
 '먹',
 '크',
 '장애',
 '들',
 '잘',
 '어서',
 '잘',
 '낳',
 '주',
 '알',
 '어야',
 '한테',
 '알',
 '냐',
 '밑',
 '진짜',
 '빠지',
 '때리',
 '어라',
 '당하',
 '으면',
 '감',
 '들',
 '때',
 '주',
 '니까',
 '줄',
 '보',
 '좀',
 '이나',
 '보',
 '흥분',
 '더라',
 '남자',
 '들',
 '시키',
 '버리',
 '좋',
 '남자',
 '으면',
 '죽',
 '짓',
 '진짜',
 '잘생기',
 '더라',
 '는데',
 '팔',
 '말',
 '미개',
 '들',
 '우리',
 '하체',
 '저러',
 '혼자',
 '어서',
 '돈',
 '이나',
 '으면',
 '좋',
 '들',
 '어야',
 '으면',
 '가지',
 '배',
 '변비',
 '똥',
 '던지',
 '진짜',
 '어도',
 '으면',
 '당하',
 '는데',
 '들',
 '좋',
 '어서',
 '낳',
 '냐',
 '진짜',
 '쓰',
 '때',
 '마다',
 '빡',
 '으면',
 '어라',
 '어서',
 '진짜',
 '어야',
 '엄마',
 '주',
 '먹',
 '냐',
 '대',
 '사',
 '자',
 '설거지',
 '까지',
 '사',
 '말',
 '들',
 '으면',
 '들',
 '어서',
 '살',
 '알',
 '보',
 '시키',
 '세',
 '밥',
 '흑인',
 '살',
 '돈',
 '받',
 '감',
 '배',
 '놓',
 '어라',
 '장사',
 '들',
 '물건',
 '불',
 '지르',
 '어야',
 '좀',
 '으면',
 '좋',
 '예쁘',
 '어떻',
 '죽이',
 '냐',
 '말']

In [ ]:
for item in temp_abuse_list :
  tem = "'"+item+"'"
  print(isOnlyThatKeyword(tem))

#왜 안나올까?

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


KeyboardInterrupt: ignored

In [ ]:
# 홍, 는데 모두 각 키워드만으로 매칭되는 실제 폭언, 성희롱 문장이 없다네요
# 즉, 사전에서 해당 두 키워드를 즉시 삭제해도, 무관하다는 뜻입니다
# 이렇게 경우의 수를 전부 계산해서 사전 csv 에서 키워드를 삭제하면 됩니다
# 단, 유의할 점은 이렇게 사전에서 키워드를 삭제하면, 다른 문장 검증시 갱신된 사전을 바탕으로 진행해야 합니다 --> 그냥 하면 미세한 확률로 에러가 날 수도! ㅎ

# 그래서 두번째 방법을 알려드립니다

print(test_set[171])
#원랜 일반 문장인데
print(predict(test_set[171][0]))
#폭언 문장으로 본다 텍스트 매칭에서 
#폭언 문장으로 가야 폭언으로 봄

['저번대선 홍이 문 개털었는데? 탄핵때문에 진거야', 0]
1


In [32]:
# 5. 문장 각색

# 예를 들어, 위의 171번 문장에서 문제가 되는게, [홍, 는데] 두개니까
example = "저번대선 김이 문 이김, 탄핵때문에 진거야"
# 이런식으로 홍,는데 를 없애서 회피하듯이 문장을 다시 짜는 겁니다

In [33]:
# 해당 문장은 문제가 없는지 검증해볼까요

def validation(sentence):
  morphs = kiwi.tokenize(example)
  flag = False
  for morph in morphs:
    if morph.form in abuse_list or morph.form in sexual_list:
      print(morph.form)
      flag = True
  
  if flag == False:
    print("문장 각색 완료")
  else:
    print("위의 키워드가 문제가 됩니다. 문장을 다시 각색하세요!")

In [34]:
validation(example)

김
이기
위의 키워드가 문제가 됩니다. 문장을 다시 각색하세요!


In [35]:
# 각색 결과 또 문제가 생기네여 ㅋㅋㅋㅋㅋㅋㅋㅋ
# 그럼 이번에는 이렇게 바꿀게여
example = "저번대선 윤이 문 개텀, 탄핵때문에 진거야"
validation(example)
predict(example)

문장 각색 완료


1

문장검색

In [ ]:
print(test_set[104])
print(predict(test_set[104][0]))

['걔네 이름부터 정 떨어짐 ㅋㅋ 평생 땅 치고 후회했으면', 0]
1


In [ ]:
#이건 포기
example = "걔네 이름부터 감명 없음 ㅋㅋ 평생 좋은 성과 내나 눈여겨 본다"
validation(example)
predict(example)

좋
보
위의 키워드가 문제가 됩니다. 문장을 다시 각색하세요!


0

두 cell만 쓴다

In [36]:
num = 577
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['형국을 알아야 너같은 놈한테도 짓밟히지 않을 거 아냐', 0]
1


In [37]:
example = "형국을 깨우치면 니부터도 안 짓밟혀"
validation(example)
print(predict(example))

문장 각색 완료
1


다시 추가 3개

In [38]:
num = 104
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['걔네 이름부터 정 떨어짐 ㅋㅋ 평생 땅 치고 후회했으면', 0]
1


In [47]:
example = "걔네 이름부터 킹받노 "
validation(example)
print(predict(example))

문장 각색 완료
1


In [48]:
num = 135
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['안락사는 죽음을 존엄하게 맞을 수 있게 돕자는 건데.', 0]
1


In [72]:
example = "강아지 유기하면 주홍글씨 남기든가 해"
validation(example)
print(predict(example))

문장 각색 완료
1


In [65]:
num = 246
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['너 아이스크림 먹고 싶다며 내가 배민으로 시켜줄까?', 0]
1


In [107]:
example = "연지곤지 분칠할라"
validation(example)
print(predict(example))

문장 각색 완료
1


In [ ]:
num = 1521
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['걔는 그 사람 잘생겼다고 지지하는 빠돌이더라', 0]
1


In [ ]:
example = "걔는 걔 훈훈하다고 사랑하는 빠돌이야"
validation(example)
print(predict(example))

문장 각색 완료
1


In [ ]:
num = 2469
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['가만히 있어 이 벌레야.', 0]
1


In [ ]:
example = "가만히 있어 이 생물아."
validation(example)
print(predict(example))

문장 각색 완료
1


In [ ]:
num = 2914
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['너 진짜 핑프같아', 0]
1


In [ ]:
example = "너 매우 핑프같아"
validation(example)
print(predict(example))

문장 각색 완료
1


In [ ]:
num = 2909
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['야 걔네도 나름 개과천선해서 열심히 한 거야', 0]
1


In [ ]:
example = "야 걔네도 나름 개과천선해 열심히 한 거야"
validation(example)
print(predict(example))

문장 각색 완료
1


In [ ]:
num = 2474
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['진짜 철딱서니가 없어도 너무 없어', 0]
1


In [ ]:
example = "무척 철딱서니가 없지만 너무 없어"
validation(example)
print(predict(example))

문장 각색 완료
1


In [ ]:
num = 2742
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['도련님 못하겠으면 아이 삼촌이라고 해라', 0]
1


In [ ]:
example = "도련님 못하면 아이 삼촌이라고 해"
validation(example)
print(predict(example))

문장 각색 완료
1


In [ ]:
num = 3626
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['나도 흑인으로 살 바엔 그냥 다음 생 도박 할 듯', 0]
1


In [ ]:
example = "나도 황인일 바엔 그냥 다음 생 다시 할 듯"
validation(example)
print(predict(example))

문장 각색 완료
1


In [ ]:
num = 3600
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['1000원에 반찬 세 개랑 밥, 국 나오면 혜자 아님?', 0]
1


In [ ]:
example = "1000원에 반찬 다섯에 찌개는 혜자 아님?"
validation(example)
print(predict(example))

문장 각색 완료
1


In [ ]:
#폭언 set에서 171, 577, 1521, 2469, 3600, 2914, 2909, 2474, 2742, 3626 총 10개 수정

In [ ]:
temp_abuse_list

['짱개',
 '어도',
 '정',
 '떨어지',
 '땅',
 '치',
 '안락사',
 '죽음',
 '맞',
 '자는',
 '홍',
 '먹',
 '시키',
 '멍',
 '때리',
 '감방',
 '사기치',
 '먹',
 '지능',
 '크',
 '넘',
 '장애',
 '사람',
 '들',
 '잘',
 '잘',
 '낳',
 '어야지',
 '놈',
 '놈',
 '놈',
 '밑',
 '빠지',
 '때리',
 '치',
 '어라',
 '가족',
 '당하',
 '사형',
 '감',
 '들',
 '내버리',
 '두',
 '차리',
 '줄',
 '닥치',
 '이나',
 '시체',
 '토막',
 '들',
 '도살',
 '시키',
 '버리',
 '좋',
 '사살',
 '부모',
 '죽',
 '멍청하',
 '짓',
 '개빡',
 '치',
 '사람',
 '지지',
 '팔',
 '자식',
 '말',
 '미개',
 '들',
 '헛',
 '소리',
 '조지',
 '걷',
 '떨어지',
 '뒤지',
 '뒤지',
 '이나',
 '뜯기',
 '좋',
 '쓰레기',
 '들',
 '참교육',
 '디지게',
 '가지',
 '똥',
 '나오',
 '미치',
 '벌레',
 '돌멩이',
 '던지',
 '어도',
 '부모',
 '자식',
 '장애인',
 '취급',
 '당하',
 '사회',
 '가난',
 '부모',
 '들',
 '좋',
 '낳',
 '쓰',
 '개',
 '빡',
 '치',
 '어라',
 '엄마',
 '차리',
 '차리',
 '먹',
 '식',
 '대',
 '모가지',
 '부리',
 '말',
 '들',
 '라도',
 '양심',
 '죄',
 '인정',
 '어야지',
 '들',
 '사회',
 '나오',
 '개',
 '반',
 '살',
 '면서',
 '맞',
 '살인',
 '사형',
 '시키',
 '개',
 '밥',
 '국',
 '나오',
 '흑인',
 '살',
 '도박',
 '주제',
 '감',
 '놓',
 '어라',
 '사람',
 '들',
 '불',
 '지르',
 '괴물',
 '길거리',
 '나오

In [ ]:
# 이번에는 문제가 없네여!
# 그럼 해당 문장을 csv에서 바꿔주면 됩니다!

# ㅎㅎ.. 복잡하죠 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
# 사전 각색 vs 문장 각색 비교하면, 아무래도 문장 각색이 신경쓸게 덜합니다  
# 사전 각색이 귀찮은 이유는 사전에서 키워드를 제거하고 갱신된 사전을 다시 코랩으로 로드하고 다음 문장을 작업해야하기 때문입니다 (갱신 안하고 작업하면, 정확한 산출이 불가)

# 그래서 왠만하면 적절하게 문장을 각색하고 검증하는 방식을 이용하면 됩니다!
# 하지만, 제가 사용한 예시 문장처럼 '는데' 이런 말도안되는 키워드가 문제가 되면, 아무래도 사전 자체에서 삭제하는 편이 좋겠죠?
# --> 판단은 알아서!, 사전 각색이 귀찮으면, 전부 문장 각색으로 하셔도 됩니다~ --> 제가 봤을 때는 '는데'만 삭제하고 이후에는 그냥 문장 각색으로 빠르게 해도 될 것 같네요

# 위의 알고리즘에서 3번부터는 폭언으로 잘 못 판별한 문장으로만 진행 중입니다
# 그러니까 성희롱으로 잘 못 판별한 문장에서도 각색이 필요해요!
# 성희롱 코드는 아래에서 확인!

# 이렇게 30문장 정도만 추가로 거르면 좋을 것 같아요! 고생하세여! ㅎ  (죄송해요ㅜ 많으면 많을 수록 좋을 것 같아요..ㅎ)
# 아 각색 과정에서, 사전을 수정했으면, 어떤 키워드를 수정했는지
# 문장을 수정했으면 어떤 문장을 수정했는지 인덱스 정보를 기록해두세요!

In [ ]:
# 부록. 성희롱으로 잘 못 판별한 문장

for rows in moral_sexual_morphs:
  abuse_matched = list()
  sexual_matched = list()

  for morph in rows[1]:
    if morph in abuse_list:
      abuse_matched.append(morph)
    if morph in sexual_list:
      sexual_matched.append(morph)

  rows.append(abuse_matched)
  rows.append(sexual_matched)

for rows in moral_sexual_morphs:
  print(rows)
  print()

# 성희롱으로 잘못 분류된 문장 모두에 대해 (test_set 인덱스, 문장 형태소, 폭언 사전 매칭 형태소, 성희롱 사전 매칭 형태소)
# 제가 봤을 때는 1102, 2078, 2267 요런 문장들이 각색하기 편해 보이네여

[25, ['성관계', '끝나', '었', '는데', '도', '계속', '이쁘', '다고', '하', '어', '주', '고', '이름', '이랑', '전화', '번호', '도', '알리', '어', '주', '네', '.'], ['알리'], ['성관계', '는데', '주', '주']]

[142, ['애견', '카페', '가', '었', '는데', '나', '의', '옆', '에서', '강아지', '들', '짝짓기', '하', 'ᆷ', '..'], ['들'], ['는데', '옆', '들', '짝짓기']]

[155, ['새', '거', '도', '아니', 'ᆫ', '스타킹', '으로', '포장', '되', '어', '있', '음'], ['새'], ['스타킹']]

[159, ['러시아', '여자', '들', '이', '리얼', '예쁘', 'ᆷ'], ['여자', '들', '예쁘'], ['여자', '들', '예쁘']]

[285, ['잘', '찍', '고', '오', '었', '지', '~', '근데', '가슴', '이', '작', '어서', '곤란', '하', '었', '음'], ['잘', '찍', '가슴', '작'], ['잘', '찍', '가슴', '작', '어서']]

[301, ['무슨', '대단', '하', 'ᆫ', '집', '아들', '이', 'ᆫ', '줄', '알', '었', '더니', '남자', '개털', '이', '구만'], ['아들', '줄'], ['무슨', '줄', '알', '남자']]

[569, ['넘', '귀엽', '지', '않', '냐', '??', '저', '정도', '이', '면', '강아지', '탈', '을', '쓰', 'ᆫ', '사람', '아니', 'ᆯ까', 'ㅋㅋ'], ['넘', '탈', '쓰', '사람'], ['냐', '탈', '쓰']]

[1091, ['ㄹㅇ', '거기', '에', '돈', '쓰', '는', '아재', '가', '얼마나', '많', '은데'], ['거기', '쓰', '아재', '많'], ['거기', '돈'

In [ ]:
num = 3532
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['그냥 다 밈이지 설마 진짜겠냐', 0]
2


In [ ]:
example = "그냥 다 밈이지 설마 리얼이겠어?"
validation(example)
print(predict(example))

문장 각색 완료
2


In [ ]:
num = 3534
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['난 단벌 숙녀야 그럴수있잔아', 0]
2


In [ ]:
example = "난 단벌 숙녀야 그럴수있잖아"
validation(example)
print(predict(example))

문장 각색 완료
2


In [ ]:
num = 2267
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['분명히 해 쨍쨍할거라 했었는데', 0]
2


In [ ]:
example = "분명히 쨍쨍할거라 했었다"
validation(example)
print(predict(example))

문장 각색 완료
2


In [ ]:
num = 1102
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['평소에 덜렁댔잖아. 너도 알면서', 0]
2


In [ ]:
example = "평소에 덜렁댔잖아. 너도 동의하잖아"
validation(example)
print(predict(example))

문장 각색 완료
2


In [ ]:
num = 2807
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['우리나라는 아직 야매가 많이서 글치 멋진데?', 0]
2


In [ ]:
example = "대한민국은 아직 야매가 다소 있지만 멋진데?"
validation(example)
print(predict(example))

문장 각색 완료
2


In [ ]:
num = 1750
print(test_set[num])
label = predict(test_set[num][0])
print(label)

['환자 덮치는 의사가 의사냐', 0]
2


In [ ]:
example = "간음하는 의사가 의사야? "
validation(example)
print(predict(example))

문장 각색 완료
2


In [ ]:
#성희롱 문장은 1102, 1750, 2807, 2267, 3534, 3532 (총 6개)

In [ ]:
# 성희롱, 폭언 두 개 합쳐 거르는게 16개입니다 